# Dynamic scaling of computations with Dagger
We run a multithreaded code and show we can dynamically spread it across many processes

In [1]:
Threads.nthreads()

4

In [4]:
using Dagger

In [5]:
code_file = tempname()*".jl" # some location where code is loaded by workers
                             # in a production environemnt this would be a package
                             # here for simplicity we use a temporary file instead
atexit(() -> rm(code_file, force=true))

In [7]:
open(code_file, "w") do f
    write(f, """
    using DataFrames, Dagger

    function get_info(i)
        sleep(0.1)
        DataFrame(;i, threadid=Threads.threadid(), procid=isdefined(Main, :Distributed) ? myid() : 0)
    end
    """)
end;

In [8]:
include(code_file)

function perfom_my_comps()
    df = DataFrame()
    append!.(Ref(df), fetch.([Dagger.@spawn get_info(i) for i in 1:10]))
    df
end

@show perfom_my_comps();

perfom_my_comps() = 10×3 DataFrame
 Row │ i      threadid  procid
     │ Int64  Int64     Int64
─────┼─────────────────────────
   1 │     1         4       0
   2 │     2         3       0
   3 │     3         1       0
   4 │     4         4       0
   5 │     5         1       0
   6 │     6         4       0
   7 │     7         3       0
   8 │     8         1       0
   9 │     9         3       0
  10 │    10         4       0


#### After having second thoughts we decide to run the code in parallel (perhaps on a cluster)

In [9]:
using Distributed

In [10]:
new_procs_xs = addprocs(2)

2-element Vector{Int64}:
 2
 3

In [11]:
# all workers need to be aware of our code
@everywhere include($code_file)

In [12]:
# Dagger needs to be notified about the new workers
Dagger.addprocs!(Dagger.Sch.eager_context(), new_procs_xs);

In [13]:
# without any changes to the code we can now run the same computation on the cluster
@show perfom_my_comps();


perfom_my_comps() = 10×3 DataFrame
 Row │ i      threadid  procid
     │ Int64  Int64     Int64
─────┼─────────────────────────
   1 │     1         4       1
   2 │     2         2       2
   3 │     3         2       3
   4 │     4         4       2
   5 │     5         1       3
   6 │     6         4       3
   7 │     7         3       1
   8 │     8         3       3
   9 │     9         1       2
  10 │    10         3       2
